## DSCC202-402 Data Science at Scale Final Project
### Tracking Tweet sentiment at scale using a pretrained transformer (classifier)
<p>Consider the following illustration of the end to end system that you will be building.  Each student should do their own work.  The project will demonstrate your understanding of Spark Streaming, the medalion data architecture using Delta Lake, Spark Inference at Scale using an MLflow packaged model as well as Exploritory Data Analysis and System Tracking and Monitoring.</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/pipeline.drawio.png">

<p>
You will be pulling an updated copy of the course GitHub repositiory: <a href="https://github.com/lpalum/dscc202-402-spring2024">The Repo</a>.  If you are unclear on how to pull an updated copy using the GitHub command line, the following <a href="https://techwritingmatters.com/how-to-update-your-forked-repository-on-github">document</a> is helpful.  Be sure to add the professors and TAs as collaborators on your project. 

- lpalum@gmail.com GitHub ID: lpalum
- ajay.anand@rochester.edu GitHub ID: ajayan12
- divyamunot1999@gmail.com GitHub ID: divyamunot
- ylong6@u.Rochester.edu GitHub ID: NinaLong2077

Once you have updates your fork of the repository you should see the following template project that is resident in the final_project directory.
</p>

<img src="https://data-science-at-scale.s3.amazonaws.com/images/notebooks.drawio.png">

<p>
You can then pull your project into the Databrick Workspace using the <a href="https://www.databricks.training/step-by-step/importing-courseware-from-github/index.html">Repos</a> feature.
Each student is expected to submit the URL of their project on GitHub with their code checked in on the main/master branch.  This illustration highlights the branching scheme that you may use to work on your code in steps and then merge your submission into your master branch before submitting.
</p>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/github.drawio.png">
<p>
Work your way through this notebook which will give you the steps required to submit a complete and compliant project.  The following illustration and associated data dictionary specifies the transformations and data that you are to generate for each step in the medallion pipeline.
</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/dataframes.drawio.png">

#### Bronze Data - raw ingest
- date - string in the source json
- user - string in the source json
- text - tweet string in the source json
- sentiment - the given sentiment of the text as determined by an unknown model that is provided in the source json
- source_file - the path of the source json file the this row of data was read from
- processing_time - a timestamp of when you read this row from the source json

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

#### Gold Data - Silver Table Inference
- timestamp - the timestamp from the silver data table rows
- mention - the mention from the silver data table rows
- cleaned_text - the cleaned_text from the silver data table rows
- sentiment - the given sentiment from the silver data table rows
- predicted_score - score out of 100 from the Hugging Face Sentiment Transformer
- predicted_sentiment - string representation of the sentiment
- sentiment_id - 0 for negative and 1 for postive associated with the given sentiment
- predicted_sentiment_id - 0 for negative and 1 for positive assocaited with the Hugging Face Sentiment Transformer

#### Application Data - Gold Table Aggregation
- min_timestamp - the oldest timestamp on a given mention (@username)
- max_timestamp - the newest timestamp on a given mention (@username)
- mention - the user (@username) that this row pertains to.
- negative - total negative tweets directed at this mention (@username)
- neutral - total neutral tweets directed at this mention (@username)
- positive - total positive tweets directed at this mention (@username)

When you are designing your approach, one of the main decisions that you will need to make is how you are going to orchestrate the streaming data processing in your pipeline.  There are several valid approaches.  First, you may choose to start the bronze_stream and let it complete (read and append all of the source data) before preceeding and starting up the silver_stream.  This approach has latency associated with it but it will allow your code to proceed in a linear fashion and process all the data by the end of your notebook execution.  Another potential approach is to start all the streams and have a "watch" method to determine when the pipeline has processed sufficient or all of the source data before stopping and displaying results.  Both of these approaches are valid and have different implications on how you will trigger your steams and how you will gate the execution of your pipeline.  Think through how you want to proceed and ask questions if you need guidance. The following references may be helpful:
- [Spark Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Autoloader - Cloudfiles](https://docs.databricks.com/en/ingestion/auto-loader/index.html)

### Be sure that your project runs end to end when *Run all* is executued on this notebook! (15 Points out of 60)

In [0]:
%run ./includes/includes

In [0]:
"""
Adding a widget to the notebook to control the clearing of a previous run.
or stopping the active streams using routines defined in the utilities notebook
"""
dbutils.widgets.removeAll()

dbutils.widgets.dropdown("clear_previous_run", "No", ["No","Yes"])
if (getArgument("clear_previous_run") == "Yes"):
    clear_previous_run()
    print("Cleared all previous data.")

dbutils.widgets.dropdown("stop_streams", "No", ["No","Yes"])
if (getArgument("stop_streams") == "Yes"):
    stop_all_streams()
    print("Stopped all active streams.")

from delta import *
dbutils.widgets.dropdown("optimize_tables", "No", ["No","Yes"])
if (getArgument("optimize_tables") == "Yes"):
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, BRONZE_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, SILVER_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, GOLD_DELTA).optimize().executeCompaction()
    print("Optimized all of the Delta Tables")

## 1.0 Import your libraries here...
- Are your shuffle partitions consistent with your cluster and your workload?
- Do you have the necessary libraries to perform the required operations in the pipeline/application?

In [0]:
# Optimize the number of post-shuffle partitions during query execution based on the data being processed.
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", "true")


In [0]:
# For machine learning models
import torch
import transformers
import mlflow
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


# For Delta tables
from delta import *

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType
from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType


import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import time


## 2.0 Use the utility functions to ...
- Read the source file directory listing
- Count the source files (how many are there?)
- print the contents of one of the files

In [0]:

# List the files in the source directory
file_list = dbutils.fs.ls(TWEET_SOURCE_PATH)
file_list

In [0]:
# Count the number of files
number_of_files = len(file_list)
print(f"Number of files in the source directory: {number_of_files}")

# Read and print the contents of the first file, if it exists
if number_of_files > 0:
    # Take the path of the first file
    first_file_path = file_list[0].path
    # Assuming the files are in text format and small enough to be collected
    file_contents = spark.read.text(first_file_path).collect()
    
    # Print the contents 
    for line in file_contents:
        print(line.value)

## 3.0 Transform the Raw Data to Bronze Data using a stream
- define the schema for the raw data
- setup a read stream using cloudfiles and the source data format
- setup a write stream using cloudfiles to append to the bronze delta table
- enforce schema
- allow a new schema to be merged into the bronze delta table
- Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defines in the includes
- name your raw to bronze stream as bronze_stream
- transform the raw data to the bronze data using the data definition at the top of the notebook

In [0]:
#  define the schema
raw_schema = StructType([
    StructField("date", StringType(), True),
    StructField("user", StringType(), True),
    StructField("text", StringType(), True),
    StructField("sentiment", StringType(), True)
])
    
raw_tweets_df = (
    spark
    .readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "json")  # Specify the format of the files in the storage
    .option("path", "s3a://voc-75-databricks-data/voc_volume/")
    .option("mergeSchema", "true")
    .schema(raw_schema)  # Apply the schema defined
    .load()  
)

bronze_data = raw_tweets_df.select (
    col('date').alias('date'),
    col('user').alias('user'),
    col('text').alias('text'),
    col('sentiment').alias('sentiment'),
    input_file_name().alias('source_file'),
    current_timestamp().alias('processing_time')
)


In [0]:
# Setup the write stream with the defined checkpoint location and table path
bronze_stream = (bronze_data
                 .writeStream
                 .format("delta")
                 .outputMode("append")
                 .option("checkpointLocation", BRONZE_CHECKPOINT)
                 .option("mergeSchema", "true")
                 .trigger(processingTime="10 seconds") 
                 .queryName("bronze_stream")
                 .start(BRONZE_DELTA)
)

In [0]:
print(bronze_stream.status)  # Check the status of stream

In [0]:
bronze_df = spark.read.format("delta").load("/tmp/labuser104917-3007334/bronze.delta")
display(bronze_df.limit(10))

## 4.0 Bronze Data Exploratory Data Analysis
- How many tweets are captured in your Bronze Table?
- Are there any columns that contain Nan or Null values?  If so how many and what will you do in your silver transforms to address this?
- Count the number of tweets by each unique user handle and sort the data by descending count.
- How many tweets have at least one mention (@) how many tweet have no mentions (@)
- Plot a bar chart that shows the top 20 tweeters (users)


In [0]:
tweet_count = bronze_df.count()
print(f"Total number of tweets: {tweet_count}")

Total number of tweets is 200000

In [0]:
# Check if there exists colunmns containing nan or null values
missing_values = bronze_df.select([count(when(isnan(col(c).cast("double")) | col(c).isNull(), c)).alias(c) for c in bronze_df.columns])
missing_values.show()

There is no nan or null values in our output. If there exists, I would consider to drop rows for sliver table.

In [0]:
tweets_by_ueser = bronze_df.groupBy('user').count().orderBy(desc('count'))
tweets_by_ueser.show()

In [0]:
bronze_mod_df = bronze_df.withColumn('mention',(col('text').contains('@').cast('integer')))
bronze_mod_df2 = bronze_mod_df.groupBy().agg(
    sum('mention').alias('num_mention'),
    (count("*") - sum('mention')).alias('num_no_mention')
)
bronze_mod_df2.show() 

The number of tweets have at least one mention is 93817, and the number of tweets have no mentions is 106183

In [0]:


# Convert Spark DataFrame to Pandas DataFrame
tweets_by_ueser_pandas = tweets_by_ueser.limit(20).toPandas()

# Plotting
plt.figure(figsize=(10, 8))
plt.bar(tweets_by_ueser_pandas['user'], tweets_by_ueser_pandas['count'])
plt.xlabel('User Name')
plt.ylabel('Number of Tweets')
plt.title('Top 20 Tweeters')
plt.xticks(rotation=45)
plt.show()

## 5.0 Transform the Bronze Data to Silver Data using a stream
- setup a read stream on your bronze delta table
- setup a write stream to append to the silver delta table
- Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
- name your bronze to silver stream as silver_stream
- transform the bronze data to the silver data using the data definition at the top of the notebook

In [0]:
def clean_mentions(df):
    """
    Removes mentions (@username) from the 'text' column and adds a new column 'cleaned_text' 
    """
    # Regular expression to find mentions
    mention_pattern = r'@\w+'
    
    # Remove mentions and store the result in a new column 'cleaned_text'
    cleaned_df = df.withColumn("cleaned_text", regexp_replace(col("text"), mention_pattern, ""))
    return cleaned_df

In [0]:
# Load Bronze delta
bronze_df = spark.readStream.format("delta").load(BRONZE_DELTA)

# Removes mentions (@username)
bronze_cleaned_df = clean_mentions(bronze_df)

silver_df = bronze_cleaned_df.select(
    col('date').alias('timestamp'),
    col('text').contains('@').cast('integer').alias('mention'),
    col('cleaned_text'), 
    col('sentiment')     
    
)
             
# Setup the stream to write to the Silver Delta table
silver_stream = (
    silver_df
    .writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", SILVER_CHECKPOINT)
    .option("path", SILVER_DELTA)
    .trigger(processingTime="10 seconds") 
    .queryName("silver_stream")  
    .start()
)

In [0]:
print(silver_stream.status)   # Check the status of stream

In [0]:
silver_data = spark.read.format("delta").load(SILVER_DELTA)
silver_data.count()

Silver data has 200000 records

## 6.0 Transform the Silver Data to Gold Data using a stream
- setup a read stream on your silver delta table
- setup a write stream to append to the gold delta table
- Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
- name your silver to gold stream as gold_stream
- transform the silver data to the gold data using the data definition at the top of the notebook
- Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
- Use a spark UDF to parallelize the inference across your silver data

In [0]:

# Load the pretrained transformer sentiment classifier
model_uri = f"models:/{MODEL_NAME}/1"
loaded_model = mlflow.pyfunc.load_model(model_uri)

@pandas_udf("score: int, label: string")
def perform_model_inference(s: pd.Series) -> pd.DataFrame:
    predictions = loaded_model.predict(s.tolist())
    return pd.DataFrame({
        "score": predictions["score"].map(lambda x: int(x * 100)).tolist(),
        "label": predictions["label"].tolist(),
    })

silver_df = spark.readStream.format("delta").load(SILVER_DELTA)
silver_df_1 = silver_df.limit(100) # For Test purpose


gold_df = (silver_df
    .withColumn("sentiment_analysis", perform_model_inference(col("cleaned_text")))
    .withColumn("predicted_score", col("sentiment_analysis.score"))  
    .withColumn("predicted_sentiment", col("sentiment_analysis.label"))
    .withColumn("sentiment_id", when(col("sentiment") == "positive", 1).otherwise(0))
    .withColumn("predicted_sentiment_id", when(col("predicted_sentiment") == "POS", 1).otherwise(0))
    .select(
        "timestamp",
        "mention",
        "cleaned_text",
        "sentiment",
        "predicted_score",
        "predicted_sentiment",
        "sentiment_id",
        "predicted_sentiment_id"
    ))

gold_stream = (
    gold_df
    .writeStream
    .format("delta")
    .option("checkpointLocation", GOLD_CHECKPOINT)
    .outputMode("append")
    .trigger(processingTime="10 seconds") 
    .queryName("gold_stream")
    .start(GOLD_DELTA)
)


In gold streaming, if the output of sentiment analysis model is neural, I set the predicted_sentiment_id as 0, just put them in the negative bucket.

In [0]:
print(gold_stream.status)  # Check the status of stream

In [0]:
gold_data = spark.read.format("delta").load(GOLD_DELTA)

In [0]:
display(gold_data.limit(10))

In [0]:
gold_data.count()

gold data has 200000 records

## 7.0 Capture the accuracy metrics from the gold table in MLflow
Store the following in an MLflow experiment run:
- Store the precision, recall, and F1-score as MLflow metrics
- Store an image of the confusion matrix as an MLflow artifact
- Store the mdoel name and the MLflow version that was used as an MLflow parameters
- Store the version of the Delta Table (input-silver) as an MLflow parameter

In [0]:


gold_data = gold_data.withColumn("predicted_sentiment_id", col("predicted_sentiment_id").cast(DoubleType()))
gold_data = gold_data.withColumn("sentiment_id", col("sentiment_id").cast(DoubleType()))

# Now create the evaluator instance
evaluator = MulticlassClassificationEvaluator(
    labelCol="sentiment_id",
    predictionCol="predicted_sentiment_id"
)

# Proceed to evaluate precision, recall, and F1 score
precision = evaluator.setMetricName("weightedPrecision").evaluate(gold_data)
recall = evaluator.setMetricName("weightedRecall").evaluate(gold_data)
f1 = evaluator.setMetricName("f1").evaluate(gold_data)
# Attempt to calculate a confusion matrix in Spark
confusion_matrix_df = gold_data.groupBy("sentiment_id").pivot("predicted_sentiment_id").count().na.fill(0)

# Convert confusion matrix to Pandas for plotting
confusion_matrix_pandas = confusion_matrix_df.toPandas()
confusion_matrix_pandas.set_index("sentiment_id", inplace=True)

# Plotting confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_matrix_pandas, annot=True, fmt="d", cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
image_path = '/tmp/confusion_matrix.png'
plt.savefig(image_path)
plt.show()

delta_table = DeltaTable.forPath(spark, SILVER_DELTA)
delta_version = delta_table.history().orderBy("version").select("version").collect()[-1][0]
# Log metrics and parameters to MLflow
with mlflow.start_run():
    mlflow.log_metric("Precision", precision)
    mlflow.log_metric("Recall", recall)
    mlflow.log_metric("F1 Score", f1)
    mlflow.log_artifact(image_path)
    mlflow.log_param("Model Name", MODEL_NAME)
    mlflow.log_param("MLflow Version", mlflow.version.VERSION)
    mlflow.log_param("Delta Table Version", delta_version)

## 8.0 Application Data Processing and Visualization
- How many mentions are there in the gold data total?
- Count the number of neutral, positive and negative tweets for each mention in new columns
- Capture the total for each mention in a new column
- Sort the mention count totals in descending order
- Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
- Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

You may want to use the "Loop Application" widget to control whether you repeateded display the latest plots while the data comes in from your streams before moving on to the next section and cleaning up your run.

*note: A mention is a specific twitter user that has been "mentioned" in a tweet with an @user reference.

In [0]:
num = gold_data.filter(gold_data['mention']==1).count()
print(f"Total number of mentions: {num}")

In [0]:

df = gold_data.withColumn('positive',when(col('sentiment') == 'positive',1).otherwise(0))\
    .withColumn('negative',when(col('sentiment') == 'negative',1).otherwise(0))

result_df = df.groupBy("mention")\
                       .agg(sum("positive").alias("positive_count"),
                            sum("negative").alias("negative_count"),
                            count("mention").alias("total_mentions"))
                       
result_df = result_df.orderBy(col("total_mentions").desc())
result_df.show()

In [0]:
# Convert to Pandas DataFrame for plotting
result_pd = result_df.toPandas()

# Top 20 positive mentions
top_positive = result_pd.nlargest(20, 'positive_count')
plt.figure(figsize=(10, 8))
top_positive.plot.bar(x='mention', y='positive_count', color='green')
plt.title('Top 20 Mentions with Positive Sentiment')
plt.xlabel('Mention')
plt.ylabel('Positive Count')
plt.show()

# Top 20 negative mentions
top_negative = result_pd.nlargest(20, 'negative_count')
plt.figure(figsize=(10, 8))
top_negative.plot.bar(x='mention', y='negative_count', color='red')
plt.title('Top 20 Mentions with Negative Sentiment')
plt.xlabel('Mention')
plt.ylabel('Negative Count')
plt.show()

## 9.0 Clean up and completion of your pipeline
- using the utilities what streams are running? If any.
- Stop all active streams
- print out the elapsed time of your notebook.

In [0]:
active_streams = spark.streams.active

# Print out the active streams
for stream in active_streams:
    print(f"Stream Name: {stream.name}")
    print(f"Is Active: {stream.isActive}")
    print(f"Stream ID: {stream.id}")
    
# Stop all active streams
for stream in active_streams:
    stream.stop()

In [0]:

# Get the notebooks ending time note START_TIME was established in the include file when the notebook started.
END_TIME = time.time()
TOTAL_TIME = END_TIME - START_TIME
print(f'It took {TOTAL_TIME} to run all the notebook')

## 10.0 How Optimized is your Spark Application (Grad Students Only)
Graduate students (registered for the DSCC-402 section of the course) are required to do this section.  This is a written analysis using the Spark UI (link to screen shots) that support your analysis of your pipelines execution and what is driving its performance.
Recall that Spark Optimization has 5 significant dimensions of considertation:
- Spill: write to executor disk due to lack of memory
- Skew: imbalance in partition size
- Shuffle: network io moving data between executors (wide transforms)
- Storage: inefficiency due to disk storage format (small files, location)
- Serialization: distribution of code segments across the cluster

Comment on each of the dimentions of performance and how your impelementation is or is not being affected.  Use specific information in the Spark UI to support your description.  

Note: you can take sreenshots of the Spark UI from your project runs in databricks and then link to those pictures by storing them as a publicly accessible file on your cloud drive (google, one drive, etc.)

References:
- [Spark UI Reference Reference](https://spark.apache.org/docs/latest/web-ui.html#web-ui)
- [Spark UI Simulator](https://www.databricks.training/spark-ui-simulator/index.html)

### ENTER YOUR MARKDOWN HERE

- Serialization: All values are 0 in result serialization time. The results of the tasks were serialized instantly, which suggests that either the result size per task was very small or the serialization logic is very efficient.Efficient serialization (and deserialization) can reduce the time and space complexity of transferring data across the network and when spilling to disk.

https://drive.google.com/file/d/1kYX5XDr0EL4q8XFuLFdSdTkFem0ePC7V/view?usp=sharing

- Spill: I couldn't find any spill information during the whole process, which means there are sufficient memory allocation to executors



- Shuffle: There is a visible Shuffle Read and Shuffle Write activity before I optimize the number of post-shuffle partitions during query execution , such as 1703.9 KiB shuffle read and 1707.1 KiB shuffle write in some tasks. This indicates that aggregations operations are causing data to be shuffled across the cluster. Shuffle operations can be costly in terms of performance because they involve disk I/O and network I/O. Optimizing transformations to reduce shuffle can help improve performance.

https://drive.google.com/file/d/16tVxfjJLAvIHsUxDGbg6eFgoAqXc30m9/view?usp=sharing

- Skew: The task at the 75th percentile processed 2 MiB/19142 records, suggesting the data distribution among tasks is skewed, with most tasks processing fewer data and a few handling more. This can lead to longer running times for tasks on busy nodes, while other nodes are not fully utilizing their computational resources.

https://drive.google.com/file/d/18E-gwBQSPHk47fiMCrLXswB3VRU1wPS8/view?usp=sharing

- Storage: The input sizes like 2.4 MiB, 1624.1 KiB, etc., are moderately sized, suggesting reasonable partitioning.Efficient storage and access patterns can significantly impact Spark performance, particularly with respect to I/O operations. Using efficient formats like Parquet can reduce serialization especially for larger datasets.

https://drive.google.com/file/d/16tVxfjJLAvIHsUxDGbg6eFgoAqXc30m9/view?usp=sharing